In [2]:
import numpy as np
import pandas as pd

In [3]:
import glob
import os
# csv_files = glob.glob(os.path.join("../data/raw/domain/suburbs", "*.csv"))
# joined_suburbs = pd.concat((pd.read_csv(f) for f in csv_files), ignore_index=True).sort_values(by='listing_id').reset_index(drop=True)

# print(len(joined_suburbs))
# joined_suburbs.head()

### Data reading

In [4]:
all_suburbs = pd.read_csv("../data/raw/domain/vic_rentals_all.csv").sort_values(by='listing_id').reset_index(drop=True)
print(len(all_suburbs))
all_suburbs.head()

12717


,listing_id,suburb,postcode,weekly_rent,bond,available_date,date_listed,days_listed,bedrooms,bathrooms,...,floorplans_count,virtual_tour,primary_type,secondary_type,agency,agency_id,agent_names,structured_features,url,land_area
0,5470976,ASCOT VALE,3032,660.0,2868.0,"Wednesday, 10 September 2025",2025-07-21,50.0,2.0,1.0,...,0.0,False,House,House,Keyhole Property Investments,8749.0,"Rentals, Reception","Air conditioning, Bath, Built in wardrobes, Cl...",https://www.domain.com.au/68-francis-street-as...,NaN
1,5604062,MELTON,3337,NaN,150.0,"Tuesday, 15 January 2008",2008-01-15,6446.0,NaN,NaN,...,0.0,False,Apartment,Apartment / Unit / Flat,Raine & Horne Melton,22328.0,NaN,NaN,https://www.domain.com.au/10-glenville-drive-m...,NaN
2,6168570,MELBOURNE,3000,310.0,1347.0,"Monday, 03 July 2023",2025-09-02,7.0,NaN,1.0,...,0.0,False,Apartment,Studio,Match Property Group,8668.0,Lisbeth Rosborg-Winter,"Built in wardrobes, Furnished, Broadband inter...",https://www.domain.com.au/32-546-flinders-stre...,NaN
3,7117948,MOONEE PONDS,3039,500.0,2173.0,"Friday, 19 September 2025",2025-09-06,3.0,2.0,1.0,...,0.0,False,Apartment,Apartment / Unit / Flat,Simone Bullen,7896.0,Ebonnie Reid,Ground floor,https://www.domain.com.au/7-64-holmes-road-moo...,NaN
4,7455074,PRAHRAN,3181,NaN,340.0,"Thursday, 22 May 2014",2012-03-02,4938.0,1.0,1.0,...,0.0,False,Apartment,Apartment / Unit / Flat,Prime Property Partners Australia,2231.0,Maia Weinberg,NaN,https://www.domain.com.au/1-60-the-avenue-prah...,NaN


### Initial feature selection

In [5]:
all_suburbs.columns

Index(['listing_id', 'suburb', 'postcode', 'weekly_rent', 'bond',
       'available_date', 'date_listed', 'days_listed', 'bedrooms', 'bathrooms',
       'carspaces', 'property_type', 'address', 'lat', 'lon', 'scraped_date',
       'domain_page_id', 'property_id', 'photo_count', 'video_count',
       'floorplans_count', 'virtual_tour', 'primary_type', 'secondary_type',
       'agency', 'agency_id', 'agent_names', 'structured_features', 'url',
       'land_area'],
      dtype='object')

In [6]:
DROPPED_COLUMNS = [
    "scraped_date",
    "domain_page_id",
    "property_id",
    "agency_id",
    "structured_features",
    "url",
]

In [7]:
all_suburbs = all_suburbs.drop(columns=DROPPED_COLUMNS)
all_suburbs.head()

,listing_id,suburb,postcode,weekly_rent,bond,available_date,date_listed,days_listed,bedrooms,bathrooms,...,lon,photo_count,video_count,floorplans_count,virtual_tour,primary_type,secondary_type,agency,agent_names,land_area
0,5470976,ASCOT VALE,3032,660.0,2868.0,"Wednesday, 10 September 2025",2025-07-21,50.0,2.0,1.0,...,144.91820,12.0,0.0,0.0,False,House,House,Keyhole Property Investments,"Rentals, Reception",NaN
1,5604062,MELTON,3337,NaN,150.0,"Tuesday, 15 January 2008",2008-01-15,6446.0,NaN,NaN,...,144.59305,6.0,0.0,0.0,False,Apartment,Apartment / Unit / Flat,Raine & Horne Melton,NaN,NaN
2,6168570,MELBOURNE,3000,310.0,1347.0,"Monday, 03 July 2023",2025-09-02,7.0,NaN,1.0,...,144.95618,1.0,0.0,0.0,False,Apartment,Studio,Match Property Group,Lisbeth Rosborg-Winter,NaN
3,7117948,MOONEE PONDS,3039,500.0,2173.0,"Friday, 19 September 2025",2025-09-06,3.0,2.0,1.0,...,144.91553,7.0,0.0,0.0,False,Apartment,Apartment / Unit / Flat,Simone Bullen,Ebonnie Reid,NaN
4,7455074,PRAHRAN,3181,NaN,340.0,"Thursday, 22 May 2014",2012-03-02,4938.0,1.0,1.0,...,144.99860,12.0,0.0,0.0,False,Apartment,Apartment / Unit / Flat,Prime Property Partners Australia,Maia Weinberg,NaN


### Feature Engineering

#### Merging with transport data

In [8]:
from transport_analysis import (
    return_stop_insights_metro_bus, 
    return_stop_insights_metro_tram, 
    return_stop_insights_metro_train, 
    return_stop_insights_regional_bus, 
    return_stop_insights_regional_train
)
RADIUS = 2 # 2km
transport_data = pd.read_csv("../data/processed/transport/transport_stops.csv")
transport_data.head()

FileNotFoundError: [Errno 2] No such file or directory: 'data/processed/transport/transport_stops.csv'

In [ ]:
# 3 mins to run
all_suburbs["num_metro_bus_stops"] = all_suburbs.apply(lambda row: return_stop_insights_metro_bus(row["lat"], row["lon"], transport_data, RADIUS), axis=1)
all_suburbs["num_metro_tram_stops"] = all_suburbs.apply(lambda row: return_stop_insights_metro_tram(row["lat"], row["lon"], transport_data, RADIUS), axis=1)
all_suburbs["num_metro_train_stops"] = all_suburbs.apply(lambda row: return_stop_insights_metro_train(row["lat"], row["lon"], transport_data, RADIUS), axis=1)
all_suburbs["num_regional_bus_stops"] = all_suburbs.apply(lambda row: return_stop_insights_regional_bus(row["lat"], row["lon"], transport_data, RADIUS), axis=1)
all_suburbs["num_regional_train_stops"] = all_suburbs.apply(lambda row: return_stop_insights_regional_train(row["lat"], row["lon"], transport_data, RADIUS), axis=1)
all_suburbs.head()

,listing_id,suburb,postcode,weekly_rent,bond,available_date,date_listed,days_listed,bedrooms,bathrooms,...,primary_type,secondary_type,agency,agent_names,land_area,num_metro_bus_stops,num_metro_tram_stops,num_metro_train_stops,num_regional_bus_stops,num_regional_train_stops
0,5470976,ASCOT VALE,3032,660.0,2868.0,"Wednesday, 10 September 2025",2025-07-21,50.0,2.0,1.0,...,House,House,Keyhole Property Investments,"Rentals, Reception",NaN,83,38,4,0,0
1,5604062,MELTON,3337,NaN,150.0,"Tuesday, 15 January 2008",2008-01-15,6446.0,NaN,NaN,...,Apartment,Apartment / Unit / Flat,Raine & Horne Melton,NaN,NaN,35,0,0,0,0
2,6168570,MELBOURNE,3000,310.0,1347.0,"Monday, 03 July 2023",2025-09-02,7.0,NaN,1.0,...,Apartment,Studio,Match Property Group,Lisbeth Rosborg-Winter,NaN,85,108,6,0,3
3,7117948,MOONEE PONDS,3039,500.0,2173.0,"Friday, 19 September 2025",2025-09-06,3.0,2.0,1.0,...,Apartment,Apartment / Unit / Flat,Simone Bullen,Ebonnie Reid,NaN,129,40,3,0,1
4,7455074,PRAHRAN,3181,NaN,340.0,"Thursday, 22 May 2014",2012-03-02,4938.0,1.0,1.0,...,Apartment,Apartment / Unit / Flat,Prime Property Partners Australia,Maia Weinberg,NaN,85,100,7,0,0


#### Merging with school location data


In [9]:
# Load processed school location data
school_data = pd.read_csv("../data/processed/schools/school_locations.csv")
school_data.head()

# Function to count schools within a radius of each property using haversine formula
from numpy import radians, cos, sin, sqrt, arctan2
def count_schools_within_radius(lat, lon, school_df, radius_km=2):
    R = 6371  # Earth radius in km
    lat1 = radians(lat)
    lon1 = radians(lon)
    lats2 = radians(school_df['Latitude'])
    lons2 = radians(school_df['Longitude'])
    dlat = lats2 - lat1
    dlon = lons2 - lon1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lats2) * sin(dlon/2)**2
    c = 2 * arctan2(sqrt(a), sqrt(1-a))
    distances = R * c
    return (distances < radius_km).sum()

# Add feature: number of schools within 2km of each property
all_suburbs['num_schools_2km'] = all_suburbs.apply(lambda row: count_schools_within_radius(row['lat'], row['lon'], school_data, radius_km=2), axis=1)
all_suburbs.head()

,listing_id,suburb,postcode,weekly_rent,bond,available_date,date_listed,days_listed,bedrooms,bathrooms,...,photo_count,video_count,floorplans_count,virtual_tour,primary_type,secondary_type,agency,agent_names,land_area,num_schools_2km
0,5470976,ASCOT VALE,3032,660.0,2868.0,"Wednesday, 10 September 2025",2025-07-21,50.0,2.0,1.0,...,12.0,0.0,0.0,False,House,House,Keyhole Property Investments,"Rentals, Reception",NaN,13
1,5604062,MELTON,3337,NaN,150.0,"Tuesday, 15 January 2008",2008-01-15,6446.0,NaN,NaN,...,6.0,0.0,0.0,False,Apartment,Apartment / Unit / Flat,Raine & Horne Melton,NaN,NaN,6
2,6168570,MELBOURNE,3000,310.0,1347.0,"Monday, 03 July 2023",2025-09-02,7.0,NaN,1.0,...,1.0,0.0,0.0,False,Apartment,Studio,Match Property Group,Lisbeth Rosborg-Winter,NaN,11
3,7117948,MOONEE PONDS,3039,500.0,2173.0,"Friday, 19 September 2025",2025-09-06,3.0,2.0,1.0,...,7.0,0.0,0.0,False,Apartment,Apartment / Unit / Flat,Simone Bullen,Ebonnie Reid,NaN,13
4,7455074,PRAHRAN,3181,NaN,340.0,"Thursday, 22 May 2014",2012-03-02,4938.0,1.0,1.0,...,12.0,0.0,0.0,False,Apartment,Apartment / Unit / Flat,Prime Property Partners Australia,Maia Weinberg,NaN,21
